In [1]:
import os
from google.cloud import storage

In [1]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'/Users/therendysuffren/Desktop/GCP Project/ServiceKey_GoogleCloud2.json'

Create Google Cloud Storage

In [22]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'/Users/therendysuffren/Desktop/GCP Project/ServiceKey_GoogleCloud2.json'
storage_client = storage.Client()
dir(storage_client)
bucket_name = 'ts_bucket2'
bucket = storage_client.bucket(bucket_name)
bucket.location = 'US'
storage_client.create_bucket(bucket)

In [ ]:
dir(storage_client)

Create a New Bucket


In [24]:
bucket_name = 'ts_bucket2'
bucket = storage_client.bucket(bucket_name)
bucket.location = 'US'
storage_client.create_bucket(bucket)

/var/folders/65/7fnhccpn1xg0d6sj8y3t7b380000gn/T/ipykernel_16812/894879367.py:3: DeprecationWarning: Assignment to 'Bucket.location' is deprecated, as it is only valid before the bucket is created. Instead, pass the location to `Bucket.create`.
  bucket.location = 'US'


<Bucket: ts_bucket2>

Function to create json object in google cloud storage

In [25]:
import json

def create_json(json_object,filename):
     # create a blob
    blob = bucket.blob(filename)
    # upload the blob 
    blob.upload_from_string(
        data=json.dumps(json_object),
        content_type='application/json'
        )
    result = filename + ' upload complete'
    return {'response' : result}

In [26]:
# your object
json_object = {
    "product": {
    "id": 1234567890,
    "title": "My Super Awesome Product",
    "vendor": "Vendor Test",
    "product_type": "Type Test",
    "created_at": "2020-05-11T16:07:45-04:00",
    "updated_at": "2020-05-26T14:32:09-04:00",
    "options": [
      {
        "id": 1234567890
      }
    ]
  }
}

In [27]:
# set the filename of your json object
filename = 'test.json'

In [28]:
# run the function and pass the json_object
print(create_json(json_object, filename))

{'response': 'test.json upload complete'}


 read the JSON file directly from the file in GCS bucket

In [29]:
import apache_beam as beam

In [31]:

from smart_open import open
import apache_beam as beam

class ReadFile(beam.DoFn):

    def __init__(self, input_path):
        self.input_path = input_path

    def start_bundle(self):
        self.client = storage.Client()
   
    def process(self, something):
        clear_data = []
        with open(self.input_path) as fin:
            for line in fin:
                data = json.loads(line)
                product = data.get('product')

                if product and product.get('id'):
                    product_id = str(product.get('id'))
                    vendor = product.get('vendor')
                    product_type = product.get('product_type')
                    updated_at = product.get('updated_at')
                    created_at = product.get('created_at')
                    product_options = product.get('options')

                    option_ids = []
                    if product_options:
                        for option in product_options:
                            option_ids.append(option.get('id'))

                    clear_data.append([product_id, vendor, product_type, updated_at, created_at, option_ids])

        yield clear_data
        #yeild clear_data passes the array we created to the next state of the pipeline.

will get as input the cleared data and write the CSV file to the output path using Pandas

In [32]:
import pandas as pd
from google.cloud import storage
class WriteCSVFIle(beam.DoFn):

    def __init__(self, bucket_name):
        self.bucket_name = bucket_name

    def start_bundle(self):
        self.client = storage.Client()

    def process(self, mylist):
        df = pd.DataFrame(mylist, columns={'product_id': str, 'vendor': str, 'product_type': str, 'updated_at': str, 'created_at': str, 'option_ids': str})

        bucket = self.client.get_bucket(self.bucket_name)
        bucket.blob(f"csv_exports.csv").upload_from_string(df.to_csv(index=False), 'text/csv')

In [33]:
import logging
import argparse

from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import SetupOptions
from apache_beam.options.pipeline_options import GoogleCloudOptions
from apache_beam.options.pipeline_options import StandardOptions





In [16]:
class DataflowOptions(PipelineOptions):

    @classmethod
    def _add_argparse_args(cls, parser):
        parser.add_argument('--input_path', type=str, default='gs://ts_bucket2/test.json')
        parser.add_argument('--output_bucket', type=str, default='didi_dataflow_temp')



    def run(argv=None):
        parser = argparse.ArgumentParser()
        known_args, pipeline_args = parser.parse_known_args(argv)

        pipeline_options = PipelineOptions(pipeline_args)
        dataflow_options = pipeline_options.view_as(DataflowOptions)

        with beam.Pipeline(options=pipeline_options) as pipeline:
            (pipeline
            | 'Start' >> beam.Create([None])
            | 'Read JSON' >> beam.ParDo(ReadFile(dataflow_options.input_path))
            | 'Write CSV' >> beam.ParDo(WriteCSVFIle(dataflow_options.output_bucket))
            )


        if __name__ == '__main__':
            logging.getLogger().setLevel(logging.INFO)
    run()


In [35]:
%%bash
python DataflowTest.py \
   --runner DataflowRunner \
   --project cobalt-ripsaw-367515 \
   --staging_location gs://ts_bucket2\
   --temp_location gs://ts_bucket2 \
   --template_location gs://ts_bucket2
   --setup_file /Users/therendysuffren/Desktop/GCP Project/setup.py\
   --save_main_session True\

bash: line 7: --setup_file: command not found


CalledProcessError: Command 'b'python DataflowTest.py \\\n   --runner DataflowRunner \\\n   --project cobalt-ripsaw-367515 \\\n   --staging_location gs://ts_bucket2\\\n   --temp_location gs://ts_bucket2 \\\n   --template_location gs://ts_bucket2\n   --setup_file /Users/therendysuffren/Desktop/GCP Project/setup.py\\\n   --save_main_session True\\\n'' returned non-zero exit status 127.